### read data

In [3]:
import pandas as pd

fd=pd.read_excel("./大病新表.xlsx",sheet_name="Sheet1")
fd.fillna(0, inplace=True)
fd.columns
# fd.dtypes
# fd.set_index(pd.to_datetime(fd['借支日期']),inplace=True)
# fd['借支日期']=(pd.to_datetime(fd['借支日期'],format='%Y%m%d'))
# fd['xi借支日期'] = fd['借支日期'].astype(str)
# fd['借支日期'] = pd.to_datetime(fd['借支日期'], format='%Y%m%d.0', errors='coerce')

# fd['借支日期'] = fd['借支日期'].astype('datetime64[ns]')
# fd.head(20)
# print("读取<<大病新表>>成功！")
# print("共计{}条记录！".format(fd.shape[0]))

Index(['科室', '住院号', '姓名', '借支日期', '借支金额', '还款日期', '还款金额', '还款日期.1', '还款金额.1',
       '还款日期.2', '还款金额.2', '批次'],
      dtype='object')

In [4]:
fd.describe()

,住院号,借支日期,借支金额,还款日期,还款日期.1,还款金额.1,还款日期.2,还款金额.2
count,1625.000000,1.625000e+03,1625.000000,1.625000e+03,1.625000e+03,1625.000000,1.625000e+03,1625.000000
mean,797985.182769,1.936772e+07,6832.186068,1.908303e+07,3.269393e+06,420.688911,6.339908e+04,7.223058
std,55754.154565,4.017591e+06,14819.411609,4.617026e+06,7.442374e+06,1654.721374,1.120202e+06,130.381596
min,459966.000000,0.000000e+00,-54825.700000,0.000000e+00,0.000000e+00,-6046.260000,0.000000e+00,0.000000
25%,785173.000000,2.020041e+07,2190.960000,2.020040e+07,0.000000e+00,0.000000,0.000000e+00,0.000000
50%,823684.000000,2.020062e+07,3702.400000,2.020062e+07,0.000000e+00,0.000000,0.000000e+00,0.000000
75%,829157.000000,2.020081e+07,6975.990000,2.020081e+07,0.000000e+00,0.000000,0.000000e+00,0.000000
max,985523.000000,2.020101e+07,269459.190000,2.020093e+07,2.020093e+07,26932.460000,2.020090e+07,3783.600000


1. filter by fd["借支金额"].isna()

In [5]:
fd[fd["借支金额"].isna()]

,科室,住院号,姓名,借支日期,借支金额,还款日期,还款金额,还款日期.1,还款金额.1,还款日期.2,还款金额.2,批次


1.1 continue to filter fd["借支金额"].isna()&fd["还款金额"].notna()


fd[fd["借支金额"].isna()&fd["还款金额"].notna()&fd["还款金额.1"].isna()]

fd[fd["借支金额"].isna()&fd["还款金额"].notna()&fd["还款金额.1"].notna()]


2. filter by fd["借支金额"].notna()

fd[fd["借支金额"].notna()]

### testing a filter

In [8]:
target=pd.read_excel("./base.xls",sheet_name="Sheet1")
target.fillna(0, inplace=True)
# target[['借支金额']].fillna(0, inplace=True)
# target[['还款金额','借支金额']]=target[['还款金额','借支金额']].fillna(0, inplace=True)
# target.info()
target.columns

Index(['科室', '住院号', '姓名', '借支日期', '借支金额', '还款日期', '还款金额', '还款日期.1', '还款金额.1',
       '还款日期.2', '还款金额.2', '批次'],
      dtype='object')

In [9]:
import process,time
result = pd.DataFrame(columns = fd.columns) # 错误结果 
matched =  pd.DataFrame(columns = fd.columns)  # 正确结果
drop_index=[] #记录需要删除对行号

max_steps=target.shape[0]
process_bar = process.ShowProcess(max_steps, 'OK')
for idx,row in target.iterrows():
    # print(type(row))
    # print(row["住院号"])
    fil=(fd["住院号"]==row["住院号"]) \
        &(fd["科室"]==row["科室"]) \
        &(fd["姓名"]==row["姓名"]) \
        &(fd["借支金额"]==row["借支金额"]) \
        &(fd["还款金额"]==row["还款金额"]) \
        &(fd["还款金额.1"]==row["还款金额.1"]) \
        &(fd["还款金额.2"]==row["还款金额.2"]) 
    _temp= fd[fil]  
    if len(_temp)==0: # can not match
        # print("{} is false".format(idx))
        result.loc[len(result)] = row
    else:
        # print("{} is true,fd index is {}".format(idx,_temp.index.tolist()))
        matched.loc[len(matched)] = row
        drop_index.extend(_temp.index.tolist())
    process_bar.show_process()
    time.sleep(0.01)
# print(result)
# write to error
result.to_excel("./result.xlsx",sheet_name="Sheet1",index=False)
matched.to_excel("./matched.xlsx",sheet_name="Sheet1",index=False)
# print(drop_index)
left=fd.drop(drop_index) # 剩余对比结果
left.to_excel("./left.xlsx",sheet_name="Sheet1",index=False)
print("对比结果如下：")
print("-----------------------------------")
print("一共对比记录："+str(target.shape[0]))
print("其中，正确比对合计："+str(matched.shape[0]))
print("错误对比结果合计："+str(result.shape[0]))
print("原始结果合计："+str(fd.shape[0]))
print("剩余对比结果合计："+str(left.shape[0]))
print("-----------------------------------")
print("最后产生三个文件：")
print("错误结果：result.xlsx")
print("正确结果：matched.xlsx")
print("剩余结果：left.xlsx")
print("-----------------------------------")

# left.shape


OK
对比结果如下：
-----------------------------------
一共对比记录：313
其中，正确比对合计：313
错误对比结果合计：0
原始结果合计：1625
剩余对比结果合计：1312
-----------------------------------
最后产生三个文件：
错误结果：result.xlsx
正确结果：matched.xlsx
剩余结果：left.xlsx
-----------------------------------


### 例子： convert data to datetime

In [ ]:
import pandas as pd

df=pd.read_excel("./base.xls",sheet_name="Sheet1")
# df.head(10)
# fd.dtypes
# fd.set_index(pd.to_datetime(fd['借支日期']),inplace=True)
# fd['借支日期']=(pd.to_datetime(fd['借支日期'],format='%Y%m%d'))
# fd['xi借支日期'] = fd['借支日期'].astype(str)
df['还款日期'] = pd.to_datetime(df['还款日期'].astype(str), format='%Y%m%d.0', errors='coerce')
df['借支日期'] = pd.to_datetime(df['借支日期'].astype(str), format='%Y%m%d.0', errors='coerce')
df
# fd['借支日期'] = fd['借支日期'].astype('datetime64[ns]')
# fd.head(20)

In [ ]:
import pandas as pd
pd.to_datetime(pd.Series([20100329.0, 20100328.0, 20100329.0]).astype(str), format='%Y%m%d.0')

In [25]:
import numpy as np
target=pd.read_excel("./base.xls",sheet_name="Sheet1")
target.fillna(0, inplace=True)
target.replace(0,np.nan,inplace=True)
target.head(10)
target.to_excel("./test.xlsx",sheet_name="Sheet1",index=False)